# voila-panel-interactive-football-pitch
This is a example widget served by `jupyter voila`. It combines `panel`, `tabular` and `bqplot` to create an interactive football pitch widget.
## Features
- Selected players on the pitch are highlighted in qgrid.
- Selected players selected in qgrid are marked on the pitch.
- Players are moveable on the pitch and their position is updated in qgrid.

In [1]:
import os
import ipywidgets as widgets

from bqplot import *
import numpy as np
import pandas as pd

import panel as pn

pn.extension('tabulator')

In [2]:
# create DataFrame for team data
columns = ['name', 'id', 'x', 'y']

tottenham_players = [
    ['Lloris', 1, 0.1, 0.5],
    ['Trippier', 2, 0.2, 0.25],
    ['Alderweireld', 4, 0.2, 0.4],
    ['Vertonghen', 5, 0.2, 0.6],
    ['D. Rose', 3, 0.2, 0.75],
    ['Sissoko', 17, 0.3, 0.4],
    ['Winks', 8, 0.3, 0.6], 
    ['Eriksen', 23, 0.4, 0.25],
    ['Alli', 20, 0.4, 0.5],
    ['Son', 7, 0.4, 0.75],
    ['H. Kane', 10, 0.45, 0.5]
]

temp_tottenham = pd.DataFrame.from_records(tottenham_players, columns=columns)
temp_tottenham['team'] = 'Tottenham Hotspur'
temp_tottenham['jersey'] = 'Blue'

liverpool_players = [
    ['Alisson', 13, 0.9, 0.5],
    ['Alexander-Arnold', 66, 0.8, 0.75],
    ['Matip', 32, 0.8, 0.6],
    ['van Dijk', 4, 0.8, 0.4],
    ['Robertson', 26, 0.8, 0.25],
    ['J. Henderson', 14, 0.7, 0.7],
    ['Fabinho', 3, 0.7, 0.5],
    ['Wijnaldum', 5, 0.7, 0.3],
    ['Salah', 11, 0.6, 0.75],
    ['Roberto Firmino', 9, 0.6, 0.5],
    ['Mané', 10, 0.6, 0.25] 
]

temp_liverpool = pd.DataFrame.from_records(liverpool_players, columns=columns)
temp_liverpool['team'] = 'FC Liverpool'
temp_liverpool['jersey'] = 'Red'

teams = pd.concat([temp_tottenham, temp_liverpool], axis=0, ignore_index=True)

In [12]:
# Define bqplot Image mark

# read pitch image
image_path = os.path.abspath('pitch.png')

with open(image_path, 'rb') as f:
    raw_image = f.read()
ipyimage = widgets.Image(value=raw_image, format='png')

scales_image = {'x': LinearScale(), 'y': LinearScale()}
axes_options = {'x': {'visible': False}, 'y': {'visible': False}}

image = Image(image=ipyimage, scales=scales_image, axes_options=axes_options)
# Full screen
image.x = [0, 1]
image.y = [0, 1]

# Define table
tabulator_widget = pn.widgets.Tabulator(teams)

In [13]:
# Define scatter plot for team data
scales={'x': LinearScale(min=0, max=1), 'y': LinearScale(min=0, max=1)}
axes_options = {'x': {'visible': False}, 'y': {'visible': False}}

team_scatter = Scatter(x=teams['x'], y=teams['y'],
                    names=teams['name'],
                    scales= scales, 
                    default_size=128,
                    interactions={'click': 'select'},
                    selected_style={'opacity': 1.0, 'stroke': 'Black'},
                    unselected_style={'opacity': 0.6},
                    axes_options=axes_options)
team_scatter.colors = teams['jersey'].values.tolist()
team_scatter.enable_move = True

# Callbacks
def change_callback(change):
    tabulator_widget.selection = change.new
        
def callback_update_tabulator(name, cell):
    new_x = round(cell['point']['x'], 2)
    new_y = round(cell['point']['y'], 2)
    
#     qgrid_widget.edit_cell(cell['index'], 'x', new_x)
#     qgrid_widget.edit_cell(cell['index'], 'y', new_y)
    tabulator_widget.patch({
    'x': [(cell['index'], new_x)],
    'y': [(cell['index'], new_y)],
})
    
    
team_scatter.observe(change_callback, names=['selected'])
team_scatter.on_drag_end(callback_update_tabulator)

In [14]:
def update(event):
#     print(event)
#     print(tabulator_widget.selected_dataframe)
    team_scatter.selected = event.new

tabulator_widget.param.watch(update, 'selection');

# Define football pitch widget
pitch_widget = Figure(marks=[image, team_scatter], padding_x=0, padding_y=0)
pitch_app = widgets.VBox([pitch_widget, pn.ipywidget(tabulator_widget)])

# Hack for increasing image size and keeping aspect ratio
width = 506.7
height = 346.7
factor = 1.8
pitch_widget.layout = widgets.Layout(width=f'{width*factor}px', height=f'{height*factor}px')

pitch_app